In [20]:
import cv2
import numpy as np
import pandas as pd
import torch
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import os
import torch

#entering_car = cv2.imread('/Users/kyungrim/fake-license-plate-detection/test_car/lightcar1.jpg')

MIN_AREA = 80
MIN_WIDTH, MIN_HEIGHT= 2, 8 # 원래 2, 8
MIN_RATIO, MAX_RATIO = 0.25, 1.0 

height, width, channel = entering_car.shape

### Get Box Info
def get_croped_image(results, image_show=True):
    boxes = results.pandas().xyxy[0]  # frame1 predictions (pandas)
    x1 = int(boxes.xmin[0])
    x2 = int(boxes.xmax[0])
    y1 = int(boxes.ymin[0])
    y2 = int(boxes.ymax[0])

    # Get Croped Image
    car_image = cv2.resize(frame[y1:y2, x1:x2], dsize=(600, 300))
    if image_show:
        plt.figure(figsize=(7, 5))
        plt.imshow(car_image)
        plt.title('원본 Croped & Resized')
        plt.show()
    return car_image

### Convert Image to Grayscale
def img_to_gray(image, image_show=True):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if image_show:
        plt.figure(figsize=(7,5))
        plt.imshow(gray, cmap='gray')
        plt.title('흑백화')
    return gray

### Adaptive Thresholding
# Thresholding을 해주기 전에 가우시안 블러를 해주는 것이 번호판을 더 잘 찾게 만들어 준다
# 가우시안 블러는 노이즈를 없애주는 작업
def img_gaussianBlur(image, image_show=True):
    img_blurred = cv2.GaussianBlur(image, ksize=(5, 5), sigmaX=0)
    # Thresholding : 지정한 threshold 값을 기준으로 이보다 낮은 값은 0, 높은 값은 255로 변환
    # 흑과 백으로만 사진을 구성하는 것 -> 이후 단계에서 Contours를 찾으려면 검은색 배경에 흰색 바탕이어야 하기 때문
    img_blur_thresh = cv2.adaptiveThreshold(
        img_blurred,
        maxValue=255.0,
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType=cv2.THRESH_BINARY_INV,
        blockSize=19,
        C=9
    )
    if image_show:
        plt.figure(figsize=(7, 5))
        plt.imshow(img_blur_thresh, cmap='gray')
        plt.title('Blur and Threshold')    
    """ 이후에 번호판 기울기 조정에 사용되는 사진은 thresholding이 되지 않은 img_thresh임"""
    """ 이게 더 번호판 인식이 잘 됨"""
    img_thresh = cv2.adaptiveThreshold(
        image,
        maxValue=255.0,
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType=cv2.THRESH_BINARY_INV,
        blockSize=19,
        C=9
    )
    
    return img_blur_thresh, img_thresh

### Find Contours
# Contours란 동일한 색 또는 동일한 강도를 가지고 있는 영역의 경계선을 연결한 선임
# findContours()는 이런 Contours를 찾는 opencv 메소드임 : 검은색 바탕에서 흰색 대상을 찾음(가우시안 블러를 적용해준 이유)
# 출력값으로 contours, hierachy를 받는데 hierachy는 쓸일이 없어 _로 받음
def find_contours(image, image_show=True):
    contours, _ = cv2.findContours(
        image,
        mode=cv2.RETR_LIST,
        method=cv2.CHAIN_APPROX_SIMPLE
    )
    contours_result = np.zeros((height, width, channel), dtype=np.uint8)
    cv2.drawContours(contours_result, contours=contours, contourIdx=-1, color=(255,255,255))
    if image_show:
        plt.figure(figsize=(7, 5))
        plt.imshow(contours_result)
        plt.title('Find Contours')
    return contours

### Prepare Data
# 찾은 Contours들의 좌표를 이용해 사각형 형태로 그려본다.
# 동시에 딕셔너리를 하나 만들어 contours들의 정보를 저장한다.
def draw_contours(contours, image_show=True):
    rectangle_result = np.zeros((height, width, channel), dtype=np.uint8)
    contours_dict = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(rectangle_result, pt1=(x,y), pt2=(x+w, y+h), color=(255,255,255), thickness=2)

        contours_dict.append({
            'contour': contour,
            'x': x,
            'y': y,
            'w': w,
            'h': h,
            'cx': x + (w / 2),
            'cy': y + (h / 2)
        })
    if image_show:
        plt.figure(figsize=(7,5))
        plt.imshow(rectangle_result, cmap='gray')
        plt.title('Draw Rectangle')  
    return contours_dict

### Select Candidates by Char Size
# 번호판 글자인 것 같은 박스들을 추려해기 위해서 w(너비)/h(높이)의 비율을 이용한다
# 아래 코드에서는 최소 비율을 0.25와 최대 비율을 1.0으로 설정한 후 기준에 맞는 박스들만 따로 저장하였다
def pickout_plate_contours01(contours_dict, image_show=True):

    possible_contours = []

    cnt = 0 # cnt로 새로운 idx를 부여
    for d in contours_dict:
        area = d['w'] * d['h']
        ratio = d['w'] / d['h']

        if area > MIN_AREA \
        and d['w'] > MIN_WIDTH and d['h'] > MIN_HEIGHT \
        and MIN_RATIO < ratio < MAX_RATIO:
            d['idx'] = cnt
            cnt += 1
            possible_contours.append(d)

    tempPlate_result = np.zeros((height, width, channel), dtype = np.uint8)
    for d in possible_contours:
        cv2.rectangle(tempPlate_result, pt1=(d['x'], d['y']), pt2=(d['x']+d['w'], d['y']+d['h']), color=(255, 255, 255), thickness=2)
    if image_show:
        plt.figure(figsize=(7, 5))
        plt.imshow(tempPlate_result, cmap='gray')
        plt.title('번호판이라 추정되는 박스1')
    return possible_contours 

### Select Candidates by Arrangement of Contours
# 남은 박스들 중에 확실하게 번호판을 찾기 위해 기준을 강화한다.
# 1. 번호판 글자 박스들은 width와 height의 비율이 모두 동일하거나 비슷하다
# 2. 번호판 글자 박스들 사이의 간격은 일정하다
# 3. 최소 3개 이상의 Contours가 인접해 있어야 한다. (대한민국 기준)
def pickout_plate_contours02(possible_contours, image_show=True):

    MAX_DIAG_MULTIPLYER = 5   # 측정 박스 대각선 길이의 5배 안에 있어야 함
    MAX_ANGLE_DIFF = 12.0     # 박스 간 중심을 이었을 때 직각삼각형이 그려지는 때 그때의 최대 기울기 theta 값
    MAX_AREA_DIFF = 0.5       # 면적의 차이
    MAX_WIDTH_DIFF = 0.8      # 너비의 차이
    MAX_HEIGHT_DIFF = 0.2     # 높이의 차이
    MIN_N_MATCHED = 3         # 위의 조건을 만족하는 경우가 3개 이상이어야 함

    def find_chars(contour_list): # 재귀 함수를 이용해 찾기 때문에 함수로 저장
        matched_result_idx = []

        for d1 in contour_list:
            matched_contours_idx = []
            for d2 in contour_list:
                if d1['idx'] == d2['idx']:
                    continue

                dx = abs(d1['cx'] - d2['cx'])
                dy = abs(d1['cy'] - d2['cy'])

                diagonal_length1 = np.sqrt(d1['w'] ** 2 + d1['h'] ** 2)

                distance = np.linalg.norm(np.array([d1['cx'], d1['cy']]) - np.array([d2['cx'], d2['cy']]))
                if dx == 0:
                    angle_diff = 90
                else:
                    angle_diff = np.degrees(np.arctan(dy / dx))
                area_diff = abs(d1['w'] * d1['h'] - d2['w'] * d2['h']) / (d1['w'] * d1['h'])
                width_diff = abs(d1['w'] - d2['w']) / d1['w']
                height_diff = abs(d1['h'] - d2['h']) / d1['h']

                if distance < diagonal_length1 * MAX_DIAG_MULTIPLYER \
                and angle_diff < MAX_ANGLE_DIFF and area_diff < MAX_AREA_DIFF \
                and width_diff < MAX_WIDTH_DIFF and height_diff < MAX_HEIGHT_DIFF:
                    matched_contours_idx.append(d2['idx'])

            matched_contours_idx.append(d1['idx'])

            if len(matched_contours_idx) < MIN_N_MATCHED:
                continue

            matched_result_idx.append(matched_contours_idx)

            unmatched_contour_idx = []
            for d4 in contour_list:
                if d4['idx'] not in matched_contours_idx:
                    unmatched_contour_idx.append(d4['idx'])

            unmatched_contour = np.take(possible_contours, unmatched_contour_idx)

            recursive_contour_list = find_chars(unmatched_contour)

            for idx in recursive_contour_list:
                matched_result_idx.append(idx)

            break

        return matched_result_idx

    result_idx = find_chars(possible_contours)

    matched_result = []
    for idx_list in result_idx:
        matched_result.append(np.take(possible_contours, idx_list))

    tempPlate_result2 = np.zeros((height, width, channel), dtype=np.uint8)

    for r in matched_result:
        for d in r:
            cv2.rectangle(tempPlate_result2, pt1=(d['x'], d['y']), pt2=(d['x']+d['w'], d['y']+d['h']), color=(255,255,255), thickness=2)
    if image_show:
        plt.figure(figsize=(7, 5))
        plt.imshow(tempPlate_result2, cmap='gray')
        plt.title('번호판이라 추정되는 박스2')
        plt.show()
        
    return matched_result

### Rotate Plate Images
# 자동차가 정방향에서 찍힌 경우면 번호판이 가지런하지만 대부분의 경우 번호판이 기울어져서 찍힘
# pytesseract를 이용하여 번호판을 인식하기 위해 번호판 부분을 정방향으로 만들어 줄 필요가 있음
# 따라서 Affine Transform을 적용하고 해당 부분만 Crop하여 출력함
def plate_affine_and_crop(matched_result, img_thresh, image_show=True):

    PLATE_WIDTH_PADDING = 1.3
    PLATE_HEIGHT_PADDING = 1.5
    MIN_PLATE_RATIO = 3
    MAX_PLATE_RATIO = 10

    plate_imgs = []
    plate_infos = []

    for i, matched_chars in enumerate(matched_result):
        sorted_chars = sorted(matched_chars, key=lambda x: x['cx'])

        plate_cx = (sorted_chars[0]['cx'] + sorted_chars[-1]['cx']) / 2
        plate_cy = (sorted_chars[0]['cy'] + sorted_chars[-1]['cy']) / 2

        plate_width = (sorted_chars[-1]['x'] + sorted_chars[-1]['w'] - sorted_chars[0]['x']) * PLATE_WIDTH_PADDING

        sum_height = 0
        for d in sorted_chars:
            sum_height += d['h']

        plate_height = int(sum_height / len(sorted_chars) * PLATE_HEIGHT_PADDING)

        triangle_height = sorted_chars[-1]['cy'] - sorted_chars[0]['cy']
        triangle_hypotenus = np.linalg.norm(
            np.array([sorted_chars[0]['cx'], sorted_chars[0]['cy']]) - 
            np.array([sorted_chars[-1]['cx'], sorted_chars[-1]['cy']])
        )

        angle = np.degrees(np.arcsin(triangle_height / triangle_hypotenus))

        rotation_matrix = cv2.getRotationMatrix2D(center=(plate_cx, plate_cy), angle=angle, scale=1.0)

        img_rotated = cv2.warpAffine(img_thresh, M=rotation_matrix, dsize=(width, height))

        img_cropped = cv2.getRectSubPix(
            img_rotated, 
            patchSize=(int(plate_width), int(plate_height)), 
            center=(int(plate_cx), int(plate_cy))
        )

        if img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO or img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO > MAX_PLATE_RATIO:
            continue

        plate_imgs.append(img_cropped)
        plate_infos.append({
            'x': int(plate_cx - plate_width / 2),
            'y': int(plate_cy - plate_height / 2),
            'w': int(plate_width),
            'h': int(plate_height)
        })
        if image_show:
            plt.figure(figsize=(7, 5)) # 원래 plt.subplot(len(matched_result), 1, i+1)
                                       # 원래 i에 따라서 몇개의 후보군이 출력되게 된다. 번호판 사진이 여러개일 수 있음
            plt.imshow(img_cropped, cmap='gray')
            plt.title('기울기 변환한 번호판')    
    return plate_imgs

### 번호판 전처리, OCR
def plate_preprocessing_OCR(plate_imgs, image_show=True):

    longest_idx, longest_text = -1, 0
    plate_chars = []

    for i, plate_img in enumerate(plate_imgs):
        plate_img = cv2.resize(plate_img, dsize=(0, 0), fx=1.6, fy=1.6)
        _, plate_img = cv2.threshold(plate_img, thresh=0.0, maxval=255.0, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        # find contours again (same as above)
        """위에 plate_img에 또 다른 후보군을 넣기 위한 작업 """
        contours, _ = cv2.findContours(plate_img, mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_SIMPLE)

        plate_min_x, plate_min_y = plate_img.shape[1], plate_img.shape[0]
        plate_max_x, plate_max_y = 0, 0

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)

            area = w * h
            ratio = w / h

            if area > MIN_AREA \
            and w > MIN_WIDTH and h > MIN_HEIGHT \
            and MIN_RATIO < ratio < MAX_RATIO:
                if x < plate_min_x:
                    plate_min_x = x
                if y < plate_min_y:
                    plate_min_y = y
                if x + w > plate_max_x:
                    plate_max_x = x + w
                if y + h > plate_max_y:
                    plate_max_y = y + h


        img_result = plate_img[plate_min_y:plate_max_y, plate_min_x:plate_max_x]
        
### Find Chars
# 추린 후보군을 이용하여 글자를 찾는다      
        img_result = cv2.GaussianBlur(img_result, ksize=(3, 3), sigmaX=0)
        _, img_result = cv2.threshold(img_result, thresh=0.0, maxval=255.0, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        img_result = cv2.copyMakeBorder(img_result, top=10, bottom=10, left=10, right=10, borderType=cv2.BORDER_CONSTANT, value=(0,0,0))

#print(pytesseract.image_to_string(img_cropped,
#    lang='cuskor',
#    config='-c tessedit_char_whitelist=0123456789가나다라마바사아자거너더러머버서어저고노도로모보소오조구누두루무부수우주하허호배 --psm 7 --oem 3'))
        
        pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'
    #    pytesseract.pytesseract.tesseract_cmd = 'D:/tesseract/tesseract.exe'
        chars = pytesseract.image_to_string(img_result, lang='kor', config='--psm 7 --oem 0')
#        chars = pytesseract.image_to_string(img_result, lang='kor', config='-c tessedit_char_whitelist=0123456789가나다라마바사아자차거너더러머버서어저고노도로모보소오조구누두루무부수우주하허호배 --psm 7 --oem 3')
        """ 자동차 번호판은 result_chars에 저장됨 """
        result_chars = ''
        has_digit = False
        for c in chars:
            if ord('가') <= ord(c) <= ord('힣') or c.isdigit():
                if c.isdigit():
                    has_digit = True
                result_chars += c

        #print(result_chars)
        plate_chars.append(result_chars)

        if has_digit and len(result_chars) > longest_text:
            longest_idx = i

        if image_show:
            plt.figure(figsize=(7, 5)) # 원래 plt.subplot(len(plate_imgs), 1, i+1)
            plt.imshow(img_result, cmap='gray')
            plt.title('Blur and Threshold Plate')
    return plate_chars

def ANPR_process(car_image):
    gray_img = img_to_gray(car_image, image_show=False)                              # 흑백화
    img_blur_thresh, img_thresh = img_gaussianBlur(gray_img, image_show=False)       # 전처리 가우시안블러
    contours = find_contours(img_blur_thresh, image_show=False)                      # Contours 찾음
    contours_dict = draw_contours(contours, image_show=False)                        # 찾은 Contours를 박스로 그림
    possible_contours = pickout_plate_contours01(contours_dict, image_show=False)    # 번호판추정 예비 박스
    matched_result = pickout_plate_contours02(possible_contours, image_show=False)   # 번호판추정 최종 박스
    plate_imgs = plate_affine_and_crop(matched_result, img_thresh, image_show=False)             # 번호판 기울기 조정, 추출
    plate_chars = plate_preprocessing_OCR(plate_imgs, image_show=False)              # 번호판 전처리, 글자추출
    return plate_chars

In [22]:
entering_car = cv2.imread('/Users/kyungrim/fake-license-plate-detection/test_car/lightcar1.jpg')
car1_chars = ANPR_process(entering_car)

entering_car = cv2.imread('/Users/kyungrim/fake-license-plate-detection/test_car/lightcar2.jpg')
car2_chars = ANPR_process(entering_car)

print("정상 차 번호판 출력 : ", car1_chars)
print("기울어진 차 번호판 출력 : ", car2_chars)

정상 차 번호판 출력 :  ['헵끄났', '157모4876', '', '', '높뽄흥효욜촬숀훗촨훗촬']
기울어진 차 번호판 출력 :  ['끄웹0', '끓훨웹쁠겜팡']
